In [1]:
!pip install pandas duckdb leveldb matplotlib seaborn tabulate ipython

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Database Performance Comparison: DuckDB, SQLite, and LMDB

This notebook demonstrates the process of creating a sample dataset, setting up DuckDB, SQLite, and LMDB databases, executing various queries, and comparing their performance with and without indexing.


## Step 1: Generate Sample Dataset

In [2]:

import pandas as pd
import random
from datetime import datetime, timedelta

# Function to generate random datetime
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

# Generate a sample dataset
data = []
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 1, 1)

for i in range(1000000):
    transaction_datetime = random_date(start_date, end_date)
    data.append([
        i,  # TransactionID (Primary Key)
        random.choice(['Customer1', 'Customer2', 'Customer3', 'Customer4', 'Customer5']),  # CustomerID
        transaction_datetime,  # TransactionDatetime
        random.choice(['Product1', 'Product2', 'Product3', 'Product4', 'Product5']),  # ProductID
        random.randint(1, 10),  # Quantity
        round(random.uniform(10, 1000), 2),  # Price
        random.choice(['Store1', 'Store2', 'Store3', 'Store4', 'Store5']),  # StoreID
        round(random.uniform(1, 50), 2)  # Discount
    ])

columns = [
    'TransactionID', 'CustomerID', 'TransactionDatetime', 'ProductID', 'Quantity', 'Price', 'StoreID', 'Discount'
]

df = pd.DataFrame(data, columns=columns)

# Display the first few rows of the dataframe to understand its structure
df.head()


,TransactionID,CustomerID,TransactionDatetime,ProductID,Quantity,Price,StoreID,Discount
0,0,Customer3,2021-03-29 20:51:58,Product1,8,87.50,Store1,17.62
1,1,Customer2,2022-09-21 06:58:15,Product3,3,889.70,Store5,45.12
2,2,Customer2,2021-03-03 03:16:56,Product3,3,912.85,Store3,1.42
3,3,Customer1,2021-09-01 07:20:19,Product5,10,821.20,Store3,23.02
4,4,Customer5,2022-12-26 22:56:51,Product3,9,78.81,Store1,40.74


#### Measure Query Time Function

In [3]:
import time

# Function to measure query time
def measure_query_time(query_func, *args):
    start_time = time.time()
    result = query_func(*args)
    end_time = time.time()
    return end_time - start_time, result


## Step 2: Setup DuckDB

In [4]:
import duckdb

# Convert necessary columns to string to avoid overflow issues
df['TransactionID'] = df['TransactionID'].astype(str)
df['CustomerID'] = df['CustomerID'].astype(str)
df['ProductID'] = df['ProductID'].astype(str)
df['StoreID'] = df['StoreID'].astype(str)

# Setup DuckDB
duckdb_conn = duckdb.connect(database=':memory:')
df.to_sql('transactions', duckdb_conn, index=False)

# Function to execute DuckDB query
def duckdb_query(query):
    return duckdb_conn.execute(query).fetchall()


## Step3: Setup SQlite

In [5]:
import sqlite3

# Setup SQLite
sqlite_conn = sqlite3.connect(':memory:')
df.to_sql('transactions', sqlite_conn, if_exists='replace', index=False)

# Function to execute SQLite query
def sqlite_query(query):
    cursor = sqlite_conn.cursor()
    cursor.execute(query)
    return cursor.fetchall()


## Step 4: Setup LevelDB

In [6]:
import leveldb

# Setup LevelDB
db = leveldb.LevelDB('leveldb_test')

# Write data to LevelDB
batch = leveldb.WriteBatch()
for idx, row in df.iterrows():
    key = str(row['TransactionID']).encode()
    value = row.to_json().encode()
    batch.Put(key, value)
db.Write(batch, sync=True)


#### Custom functions to execute LevelDB query with key identifiers


In [7]:

def leveldb_query_single(id, store_id=None):
    results = []
    try:
        value = db.Get(str(id).encode())
        row = pd.read_json(value.decode(), typ='series')
        if store_id is None or row['StoreID'] == store_id:
            results.append(row)
    except KeyError:
        pass
    return results

def leveldb_query_multiple(ids):
    results = []
    for id in ids:
        try:
            value = db.Get(str(id).encode())
            row = pd.read_json(value.decode(), typ='series')
            results.append(row)
        except KeyError:
            pass
    return results

# Function to execute aggregation in LevelDB
def leveldb_query_aggregation():
    results = {}
    for key, value in db.RangeIter():
        row = pd.read_json(value.decode(), typ='series')
        customer_id = row['CustomerID']
        total_spent = row['Price'] * row['Quantity']
        if customer_id in results:
            results[customer_id] += total_spent
        else:
            results[customer_id] = total_spent
    return results


## Example Queries to Test

In [8]:
# Function to generate a query with 30 random TransactionIDs
def generate_random_keys_query():
    random_transaction_ids = random.sample(range(1, 10001), 30)  # Assuming TransactionID ranges from 1 to 10000
    transaction_id_str = ', '.join(f"'{id}'" for id in random_transaction_ids)
    return f"SELECT * FROM transactions WHERE TransactionID IN ({transaction_id_str})"

# Example queries to test
def get_queries():
    return [
        "SELECT * FROM transactions WHERE TransactionID = '1'",  # Query by ID
        "SELECT * FROM transactions WHERE TransactionID = '1' AND StoreID = 'Store1'",  # Query by ID and another column
        generate_random_keys_query(),  # Query with multiple keys
        "SELECT CustomerID, SUM(Price * Quantity) FROM transactions GROUP BY CustomerID"  # Aggregation query
    ]


# LevelDB conditions
leveldb_conditions = [
    (1, None),
    (1, 'Store1'),
]

# Function to run queries multiple times and return the average time
def average_query_time(query_func, query_generator, runs=100):
    total_time = 0
    for _ in range(runs):
        query = query_generator() if callable(query_generator) else query_generator
        elapsed_time, _ = measure_query_time(query_func, query)
        total_time += elapsed_time
    return total_time / runs

# Function to run leveldb queries multiple times and return the average time
def average_leveldb_time(query_func, args, runs=1):
    total_time = 0
    for _ in range(runs):
        elapsed_time, _ = measure_query_time(query_func, *args)
        total_time += elapsed_time
    return total_time / runs

# Measure query times
queries = get_queries()
duckdb_times = [average_query_time(duckdb_query, q) for q in queries]
sqlite_times = [average_query_time(sqlite_query, q) for q in queries]

leveldb_times = [
    average_leveldb_time(leveldb_query_single, leveldb_conditions[0]),
    average_leveldb_time(leveldb_query_single, leveldb_conditions[1]),
    average_leveldb_time(leveldb_query_multiple, [random.sample(range(1, 10001), 30)]),
    average_leveldb_time(leveldb_query_aggregation, [])
]


## Add Indexing and Run Queries Again

In [12]:

# Add indexing for DuckDB and SQLite and measure again
# duckdb_conn.execute("CREATE INDEX idx_transaction_id ON transactions (TransactionID)")
# sqlite_conn.execute("CREATE INDEX idx_transaction_id ON transactions (TransactionID)")

duckdb_times_indexed = [average_query_time(duckdb_query, q) for q in queries]
sqlite_times_indexed = [average_query_time(sqlite_query, q) for q in queries]


## Print Results

In [16]:
# Function to format times with 8 decimal places
def format_times(times):
    return [f"{t:.8f}" for t in times]

# Print results
print("DuckDB Times (without index):", format_times(duckdb_times))
print("SQLite Times (without index):", format_times(sqlite_times))
print("LevelDB Times (without index):", format_times(leveldb_times))
print("DuckDB Times (with index):", format_times(duckdb_times_indexed))
print("SQLite Times (with index):", format_times(sqlite_times_indexed))

DuckDB Times (without index): ['0.00136316', '0.00226011', '0.00696233', '0.00561259']
SQLite Times (without index): ['0.03519033', '0.03502800', '0.09809309', '0.34471523']
LevelDB Times (without index): ['0.00110912', '0.00069404', '0.02440310', '668.97385502']
DuckDB Times (with index): ['0.00069448', '0.00026879', '0.00718518', '0.00516721']
SQLite Times (with index): ['0.00000274', '0.00000168', '0.00004944', '0.32978075']


In [17]:
import pandas as pd

# Format times for the table
def format_times(times):
    return [f'{time:.6f} seconds' for time in times]

# Data for the table
data = {
    'Query Type': [
        'Query by ID',
        'Query by ID with Column',
        'Query with Multiple Keys',
        'Aggregation Query'
    ],
    'DuckDB (without index)': format_times(duckdb_times),
    'SQLite (without index)': format_times(sqlite_times),
    'LevelDB (without index)': format_times(leveldb_times),
    'DuckDB (with index)': format_times(duckdb_times_indexed),
    'SQLite (with index)': format_times(sqlite_times_indexed)
}

# Create a DataFrame
df = pd.DataFrame(data)

# Function to convert DataFrame to Markdown table format
def dataframe_to_markdown(df):
    return df.to_markdown(index=False, tablefmt='pipe')

# Convert the DataFrame to Markdown table
markdown_table = dataframe_to_markdown(df)

# Display the table
print(markdown_table)


| Query Type               | DuckDB (without index)   | SQLite (without index)   | LevelDB (without index)   | DuckDB (with index)   | SQLite (with index)   |
|:-------------------------|:-------------------------|:-------------------------|:--------------------------|:----------------------|:----------------------|
| Query by ID              | 0.001363 seconds         | 0.035190 seconds         | 0.001109 seconds          | 0.000694 seconds      | 0.000003 seconds      |
| Query by ID with Column  | 0.002260 seconds         | 0.035028 seconds         | 0.000694 seconds          | 0.000269 seconds      | 0.000002 seconds      |
| Query with Multiple Keys | 0.006962 seconds         | 0.098093 seconds         | 0.024403 seconds          | 0.007185 seconds      | 0.000049 seconds      |
| Aggregation Query        | 0.005613 seconds         | 0.344715 seconds         | 668.973855 seconds        | 0.005167 seconds      | 0.329781 seconds      |
